In [72]:
# CODE FOR LOADING SAVED CHATBOT

# import libraries
import random
import json
import pickle
import numpy as np
import nltk
from keras.models import load_model
from nltk.stem import WordNetLemmatizer

# load intents and saved model output files
lemmatizer = WordNetLemmatizer()
intents = json.loads(open("intents.json").read())
words = pickle.load(open('words.pkl', 'rb'))
labels = pickle.load(open('labels.pkl', 'rb'))
model = load_model('chatbotmodel.h5')

# separate input words from input sentence
def clean_up_sentences(sentence):
	sentence_words = nltk.word_tokenize(sentence)
	sentence_words = [lemmatizer.lemmatize(word)
					for word in sentence_words]
	return sentence_words

# determine if input words in saved model words list
def bagw(sentence):
	sentence_words = clean_up_sentences(sentence)
	bag = [0]*len(words)
	for w in sentence_words:
		for i, word in enumerate(words):
			if word == w:
				bag[i] = 1
	return np.array(bag)

# predict label of sentence based on input words
def predict_class(sentence):
	bow = bagw(sentence)
	res = model.predict(np.array([bow]))[0]
	ERROR_THRESHOLD = 0.25
	results = [[i, r] for i, r in enumerate(res)
			if r > ERROR_THRESHOLD]
	results.sort(key=lambda x: x[1], reverse=True)
	return_list = []
	for r in results:
		return_list.append({'intent': labels[r[0]],
							'probability': str(r[1])})
		return return_list

# print random response from appropriate responses for label
def get_response(intents_list, intents_json):
	tag = intents_list[0]['intent']
	list_of_intents = intents_json['intents']
	result = ""
	for i in list_of_intents:
		if i['tag'] == tag:
			result = random.choice(i['responses'])
			break
	return result

# create infinite loop that prompts for input
print("Chatbot live:")

while True:
	message = input("")
	ints = predict_class(message)
	res = get_response(ints, intents)
	print(res)


Chatbot live:
hello
1/1 [==============================] - 0s 57ms/step
Hey! How can I assist you today?
thank you
1/1 [==============================] - 0s 36ms/step
You're welcome!
recommend
1/1 [==============================] - 0s 20ms/step
Based on your previous transaction you may save upto


KeyboardInterrupt: ignored